In [1]:
import numpy as np
import pandas as pd

import matplotlib
from matplotlib import patheffects as pe
import matplotlib.pyplot as plt
import seaborn as sns
import squarify

import regex as re
import string

import tqdm
from  pprint import pprint

import fitz  ### it is PyMuPDF

from collections import defaultdict, namedtuple

plt.rcParams["figure.figsize"] = (15, 10)
pd.set_option('display.max_colwidth', 100)

## Read CSVs

In [19]:
webpage_quotes, zajka_proverbs, bnt_proverbs =  map(pd.read_csv, ['data/webpage_data/all_quotes.csv',
                                                                  'data/book_data/after_formatting_Прыказкі_Прымаўкі_З_Косаўшчыны.csv', 
                                                                  'data/book_data/prefiltered_Беларуская_народная_творчасць.csv'])

In [20]:
webpage_quotes.head(3)

,quote,author,source,low_text
0,"І будзе ўнукаў панаванне\nТам, дзе сягоння пла...",Янка Купала,nlb.by,і будзе ўнукаў панаванне там дзе сягоння плача...
1,Няма для духа вольнага граніцы,Янка Купала,nlb.by,няма для духа вольнага граніцы
2,"Песні, знайце, – сэрца дар",Янка Купала,nlb.by,песні знайце – сэрца дар


In [5]:
bnt_proverbs.head(3)

,category,proverb
0,"ПРЫРОДА, ЯЕ СТЫХІІ I З'ЯВЫ. ЖЫВЕЛЬНЫ I РАСЛІНН...","Зямля — маці, а праца — бацька."
1,ПОРЫ ГОДУ. НАРОДНЫ КАЛЯНДАР,Жураўлі ляцяць высока — зіма яшчэ далека.
2,СЕЛЬСКАЯ ГАСПАДАРКА,"Пабыла Хімка ў лядах, пабачыла працы."


In [4]:
zajka_proverbs.head(3)

,page,category,proverb
0,11,ПРЫКАЗКІ І ПРЫМАЎКІ,"Абедаў, а жывот не ведаў. \nГэтак кажа той, хт..."
1,11,ПРЫКАЗКІ І ПРЫМАЎКІ,"Або пан, або прапаў. \nТак кажуць, калі пачына..."
2,11,ПРЫКАЗКІ І ПРЫМАЎКІ,"Абы балота, а чэрці знойдуцца. \n“Былі б рукі,..."


## Align CSVs to single format

#### webpage_quotes

In [21]:
webpage_quotes = webpage_quotes.rename(columns={'quote': 'phrase', 'author': 'category'}).drop('low_text', axis=1)
webpage_quotes['source_type'] = 'webpage'
webpage_quotes.head(2)

,phrase,category,source,source_type
0,"І будзе ўнукаў панаванне\nТам, дзе сягоння пла...",Янка Купала,nlb.by,webpage
1,Няма для духа вольнага граніцы,Янка Купала,nlb.by,webpage


In [23]:
webpage_quotes.source.value_counts()

dumki.org                      726
nlb.by                         102
svaboda.org_a_24255332.html     86
Name: source, dtype: int64

In [24]:
def set_precise_webpage_source(source):
    source_map = {
        'dumki.org': 'dumki.org',
        'nlb.by': "https://www.nlb.by/by/infarmatsyynyya-resursy/elektronnyya-infarmatsyynyya-resursy/resursy-natsyyanalnay-bibliyateki-belarusi/virtualnyya-praekty-vysta-ki-i-kalektsyi/virtualnyya-praekty-bibliyateki/klasiki-susvetnay-litaratury-yanka-kupala-i-yakub-/vyslo-i-vykazvanni-afaryzmy-belaruskikh-pesnyaro",
        'svaboda.org_a_24255332.html': r'https://www.svaboda.org/a/24255332.html'
    }
    return source_map[source]
webpage_quotes['full_source'] = webpage_quotes.source.apply(set_precise_webpage_source)

In [26]:
webpage_quotes.source = webpage_quotes.source.apply(lambda x: "svaboda.org" if x == "svaboda.org_a_24255332.html" else x)
webpage_quotes.source.value_counts()

dumki.org      726
nlb.by         102
svaboda.org     86
Name: source, dtype: int64

In [27]:
webpage_quotes.head(2)

,phrase,category,source,source_type,full_source
0,"І будзе ўнукаў панаванне\nТам, дзе сягоння пла...",Янка Купала,nlb.by,webpage,https://www.nlb.by/by/infarmatsyynyya-resursy/...
1,Няма для духа вольнага граніцы,Янка Купала,nlb.by,webpage,https://www.nlb.by/by/infarmatsyynyya-resursy/...


---
#### bnt phrases

In [28]:
bnt_proverbs = bnt_proverbs.rename(columns={'proverb': 'phrase'})
bnt_proverbs['source'] = 'Беларуская народная творчасць. Прыказкі і прымаўкі. У дзьвюх кнігах'
bnt_proverbs.head(2)

,category,phrase,source
0,"ПРЫРОДА, ЯЕ СТЫХІІ I З'ЯВЫ. ЖЫВЕЛЬНЫ I РАСЛІНН...","Зямля — маці, а праца — бацька.",Беларуская народная творчасць. Прыказкі і прым...
1,ПОРЫ ГОДУ. НАРОДНЫ КАЛЯНДАР,Жураўлі ляцяць высока — зіма яшчэ далека.,Беларуская народная творчасць. Прыказкі і прым...


In [29]:
bnt_proverbs['full_source'] = bnt_proverbs['source']
bnt_proverbs['source_type'] = 'book'
bnt_proverbs.head(2)

,category,phrase,source,full_source,source_type
0,"ПРЫРОДА, ЯЕ СТЫХІІ I З'ЯВЫ. ЖЫВЕЛЬНЫ I РАСЛІНН...","Зямля — маці, а праца — бацька.",Беларуская народная творчасць. Прыказкі і прым...,Беларуская народная творчасць. Прыказкі і прым...,book
1,ПОРЫ ГОДУ. НАРОДНЫ КАЛЯНДАР,Жураўлі ляцяць высока — зіма яшчэ далека.,Беларуская народная творчасць. Прыказкі і прым...,Беларуская народная творчасць. Прыказкі і прым...,book


---
#### zajka-cosaushchyna phrases

In [67]:
zajka_proverbs.rename(columns={'proverb': 'phrase'}, inplace=True)
zajka_proverbs['source'] = 'Алесь Зайка. Прыказкі і прымаўкі з Косаўшчыны'
zajka_proverbs['full_source'] = zajka_proverbs.apply(lambda x: x.source + ', page ' + str(x.page), axis=1)
zajka_proverbs.head(2)

,page,category,phrase,source,full_source
0,11,ПРЫКАЗКІ І ПРЫМАЎКІ,"Абедаў, а жывот не ведаў. \nГэтак кажа той, хт...",Алесь Зайка. Прыказкі і прымаўкі з Косаўшчыны,"Алесь Зайка. Прыказкі і прымаўкі з Косаўшчыны,..."
1,11,ПРЫКАЗКІ І ПРЫМАЎКІ,"Або пан, або прапаў. \nТак кажуць, калі пачына...",Алесь Зайка. Прыказкі і прымаўкі з Косаўшчыны,"Алесь Зайка. Прыказкі і прымаўкі з Косаўшчыны,..."


In [68]:
zajka_proverbs.full_source.value_counts()

Алесь Зайка. Прыказкі і прымаўкі з Косаўшчыны, page 183    21
Алесь Зайка. Прыказкі і прымаўкі з Косаўшчыны, page 166    20
Алесь Зайка. Прыказкі і прымаўкі з Косаўшчыны, page 188    19
Алесь Зайка. Прыказкі і прымаўкі з Косаўшчыны, page 169    19
Алесь Зайка. Прыказкі і прымаўкі з Косаўшчыны, page 170    19
                                                           ..
Алесь Зайка. Прыказкі і прымаўкі з Косаўшчыны, page 286     6
Алесь Зайка. Прыказкі і прымаўкі з Косаўшчыны, page 162     5
Алесь Зайка. Прыказкі і прымаўкі з Косаўшчыны, page 129     5
Алесь Зайка. Прыказкі і прымаўкі з Косаўшчыны, page 156     3
Алесь Зайка. Прыказкі і прымаўкі з Косаўшчыны, page 136     2
Name: full_source, Length: 276, dtype: int64

In [69]:
zajka_proverbs['source_type'] = 'book'

In [70]:
zajka_proverbs.drop('page', axis=1, inplace=True)
zajka_proverbs.head(2)

,category,phrase,source,full_source,source_type
0,ПРЫКАЗКІ І ПРЫМАЎКІ,"Абедаў, а жывот не ведаў. \nГэтак кажа той, хт...",Алесь Зайка. Прыказкі і прымаўкі з Косаўшчыны,"Алесь Зайка. Прыказкі і прымаўкі з Косаўшчыны,...",book
1,ПРЫКАЗКІ І ПРЫМАЎКІ,"Або пан, або прапаў. \nТак кажуць, калі пачына...",Алесь Зайка. Прыказкі і прымаўкі з Косаўшчыны,"Алесь Зайка. Прыказкі і прымаўкі з Косаўшчыны,...",book


---
## All data df

In [79]:
full_df = pd.concat([webpage_quotes, bnt_proverbs, zajka_proverbs],
          axis=0,
          ignore_index=True)
full_df.sample(2)

,phrase,category,source,source_type,full_source
8779,Раззёва! \nРазява.,ЗНЕВАЖАННІ,Алесь Зайка. Прыказкі і прымаўкі з Косаўшчыны,book,"Алесь Зайка. Прыказкі і прымаўкі з Косаўшчыны,..."
5604,"Мужык гол-гоп, а яго хлоп-хлоп.","КАРОЛЬ, ЦАР. ПАНЫ, ПАДПАНКІ, ШЛЯХТА. ПРЫГОН, М...",Беларуская народная творчасць. Прыказкі і прым...,book,Беларуская народная творчасць. Прыказкі і прым...


In [86]:
full_df.shape

(9749, 5)

#### duplicates search 

In [100]:
from thefuzz import fuzz, process
from tqdm import tqdm_pandas
from tqdm.notebook import trange, tqdm
# from tqdm.notebook import tqdm_pandas

In [101]:
tqdm.pandas()

In [166]:
# THIS DOES NOT WORK FOR OUR CASE
#token_set_sample = full_df.phrase.apply(lambda x: process.extractBests(x, full_df.phrase.values, score_cutoff=80, scorer=fuzz.token_set_ratio))
def preprocess(txt):
    return txt.strip().lower() 

token_sort_sample = full_df.phrase.apply(lambda x: process.extractBests(preprocess(x), full_df.phrase.values, processor=preprocess, score_cutoff=85, scorer=fuzz.token_sort_ratio))

In [168]:
NUM = len(token_sort_sample)
higher_threshold = 90

count = 0
any_close = False
for sort_sample in token_sort_sample[:NUM+1]:
    any_close = False
    
    if len(sort_sample) > 1:
        for (txt, similar) in sort_sample[1:]:
            if similar >= higher_threshold:
                print(f'{similar=}')
                print(txt)
                any_close = True
        
        if any_close:
            print(f'Original text: {sort_sample[0][0]}')
            print('*' * 20)
            count += 1

print(count)

similar=94
Воля ‒ сонейку роўня
Original text: Воля сонейку раўня
********************
similar=98
Якім бы велічным не здаваўся трон, усё роўна на ім сядзіць задніца
Original text: Якім бы велічным ні здаваўся трон, усё роўна на ім сядзіць задніца.
********************
similar=100
Кожны народ мае хаця б адзін геніяльны твор, і гэты твор ‒ мова
Original text: Кожны народ мае хаця б адзін геніяльны твор, і гэты твор - мова.
********************
similar=100
Час ‒ люстэрка, у якім кожны ўбачыць сябе, як іншага
Original text: Час - люстэрка, у якім кожны ўбачыць сябе, як іншага.
********************
similar=100
Абыякавы да мінулага ня мае аніякай інтэлектуальнай перавагі над жывёлай, і таму ёсьць першы кандыдат на маральную, а затым і фізычную сьмерць. Усё адно хто гэта ‒ чалавек ці народ
Original text: Абыякавы да мінулага ня мае аніякай інтэлектуальнай перавагі над жывёлай, і таму ёсьць першы кандыдат на маральную, а затым і фізычную сьмерць. Усё адно хто гэта - чалавек ці народ.
*********

In [169]:
full_df[full_df.phrase.str.contains('Быў бы хлеб')]#['full_source']

,phrase,category,source,source_type,full_source
3379,"Быў бы хлеб, а мышы будуць.",МАТЭРЫЯЛЬНЫ БЫТ. ГАСПАДАРЛІВАСЦЬ. СЯЛЯНСКАЯ СЯДЗІБА,Беларуская народная творчасць. Прыказкі і прымаўкі. У дзьвюх кнігах,book,Беларуская народная творчасць. Прыказкі і прымаўкі. У дзьвюх кнігах
5291,"Быў бы хлеб, а мышы будуць.",ХАРЧАВАННЕ,Беларуская народная творчасць. Прыказкі і прымаўкі. У дзьвюх кнігах,book,Беларуская народная творчасць. Прыказкі і прымаўкі. У дзьвюх кнігах


## Show some statistics on data

In [ ]:
full_df